Strongly based on:
http://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/

In [1]:
import keras
import numpy as np

from keras.layers import (
    Dense,
    Flatten,
    Conv2D,
    MaxPooling2D
)
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
IMG_DIM = 227
LABELS = [
    'bb',
    'bk',
    'bn',
    'bp',
    'bq',
    'br',
    'empty',
    'wb',
    'wk',
    'wn',
    'wp',
    'wq',
    'wr'
]

NUM_CLASSES = len(LABELS)
BATCH_SIZE = 128
EPOCHS = 10

Funções para amostragem aleatória:

In [4]:
def randomsampling(X,Y):
    size = X.shape[0]
    idx = np.random.choice(np.arange(X.shape[0]), X.shape[0], replace=False)
    x_sample = X[idx]
    y_sample = Y[idx]
    return x_sample, y_sample

def conta_label(Y,label):
    count= 0
    for i in range(len(Y)):
        if Y[i] == label:
            count+=1
    return count

def conta_todos_labels(Y):
    labels = []
    for j in range(len(LABELS)):
        labels =labels + [conta_label(Y,j)]
    return labels

#label_count = np.array(conta_todos_labels(Y)) # numero de amostras de cada label

def amostragem_aleatoria(X,Y,k):
    x=[]
    y=[]
    prop = k/10360
    l_c_prop = np.round(label_count*prop)
    for i in range(len(LABELS)):
        n = []
        j = 0
        while len(n) != (l_c_prop[i]):
            if Y[j] == i:
                n += [X[j]]
                j += 1
                y += [i]
            else:
                j += 1
        x+=n
    x = np.array(x)
    y = np.array(y)
    x, y = randomsampling(x,y)
    return x,y

In [5]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [6]:
def make_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(7, 7), strides=(1, 1), activation='relu', input_shape=(IMG_DIM, IMG_DIM, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, kernel_size=(7, 7)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    model.compile(
        loss=keras.losses.categorical_crossentropy,
        optimizer=keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    return model


In [7]:
def prepare_samples(X, Y):
    print('X shape: ', X.shape)
    x = X.reshape(X.shape[0], IMG_DIM, IMG_DIM, 1).astype('float32')
    # Normalization
    x /= 255
    y = keras.utils.to_categorical(Y, NUM_CLASSES)
    return (x, y)

In [8]:
def train(model, X, y, X_test=None, y_test=None):
    x_train, y_train = prepare_samples(X, y)
    x_val = None
    y_val = None
    if X_test is not None and y_test is not None:
        x_val, y_val = prepare_samples(X_test, y_test)
    history = AccuracyHistory()
    model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(x_val, y_val),
        callbacks=[history]
    )
    return history

In [9]:
def score(model, X, y):
    x_test, y_test = prepare_samples(X, y)
    return model.evaluate(x_test, y_test, verbose=0)

In [10]:
from data_config import DataConfig

data = DataConfig('data.config')
train_data, train_labels = data.get_train()
test_data, test_labels = data.get_test()
test_labels = test_labels[:, 0].astype(int).reshape(-1,1)
train_labels = train_labels[:, 0].astype(int).reshape(-1,1)
label_count = np.array(conta_todos_labels(train_labels)) # numero de amostras de cada label
print('train_data shape: ', train_data.shape)
print('test_data shape: ', test_data.shape)
print('numero de amostras de cada label: ', label_count)

train_data shape:  (10360, 227, 227)
test_data shape:  (740, 227, 227)
numero de amostras de cada label:  [ 532  279  502 1471  369  271 3461  517  279  512 1484  371  312]


In [11]:
print('Creating CNN model...')
chess_model = make_model()

Creating CNN model...


In [ ]:
print('Training...')
history = train(chess_model, train_data, train_labels, test_data, test_labels)
print('Evaluating...')
score = score(chess_model, test_data, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Done.')

Amostragem aleatória

In [12]:
X,Y = amostragem_aleatoria(train_data,train_labels,1000) # X,Y e numero de amostras

In [13]:
Y[:100]

array([10,  6, 10, 10,  6,  3,  7, 10,  6,  9, 10, 10,  3,  5, 11,  5,  6,
        5,  7, 10, 10,  2,  1, 10,  7,  6,  6,  6,  1,  9,  0,  7,  6, 12,
        6, 10,  6,  6,  3,  0,  8,  6,  2,  6,  3,  3,  0,  9,  6,  6,  6,
        6,  6,  4,  7, 10, 10,  6,  3,  3,  3,  3,  9,  6, 12, 10, 10, 11,
        3,  6, 10,  6,  1,  6,  6,  6,  6,  9,  6,  6, 11,  9,  0,  3, 12,
        8,  3,  6,  9,  3,  4,  0,  6,  6,  9,  6,  1,  6,  6,  6])